# Quaternion Visualizernumpy scipy matplotlib ipywidgets plotly


In [11]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import serial
import numpy as np

# ----------------- SERIAL CONFIG -----------------
ser = serial.Serial('COM8', 115200, timeout=1)

# ----------------- CUBE DATA -----------------
vertices = [
    [-1, -1, -1],
    [-1,  1, -1],
    [ 1,  1, -1],
    [ 1, -1, -1],
    [-1, -1,  1],
    [-1,  1,  1],
    [ 1,  1,  1],
    [ 1, -1,  1]
]

faces = [
    [0, 1, 2, 3],
    [4, 5, 6, 7],
    [0, 4, 7, 3],
    [1, 5, 6, 2],
    [0, 1, 5, 4],
    [3, 2, 6, 7]
]

colors = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (1, 1, 0),
    (1, 0, 1),
    (0, 1, 1)
]

def draw_cube():
    glBegin(GL_QUADS)
    for i, face in enumerate(faces):
        glColor3fv(colors[i])
        for vertex in face:
            glVertex3fv(vertices[vertex])
    glEnd()

def init_lighting():
    glEnable(GL_DEPTH_TEST)
    glDisable(GL_LIGHTING)       # <-- turn off lighting
    glDisable(GL_COLOR_MATERIAL) # <-- make sure color is unaffected by lights
    glShadeModel(GL_FLAT)        # or GL_SMOOTH, doesn't matter now


def read_quaternion():
    try:
        line = ser.readline().decode(errors="ignore").strip()
        if line:
            parts = line.split(',')
            if len(parts) == 4:
                return [float(p) for p in parts]
    except:
        pass
    return None

def quaternion_to_matrix(q):
    qw, qx, qy, qz = q
    # Normalize just in case
    norm = np.linalg.norm(q)
    if norm == 0:
        return np.identity(4, dtype=np.float32)
    qw, qx, qy, qz = qw/norm, qx/norm, qy/norm, qz/norm

    # Convert quaternion to rotation matrix (column-major for OpenGL)
    m = np.identity(4, dtype=np.float32)
    m[0][0] = 1 - 2*qy*qy - 2*qz*qz
    m[0][1] = 2*qx*qy - 2*qz*qw
    m[0][2] = 2*qx*qz + 2*qy*qw

    m[1][0] = 2*qx*qy + 2*qz*qw
    m[1][1] = 1 - 2*qx*qx - 2*qz*qz
    m[1][2] = 2*qy*qz - 2*qx*qw

    m[2][0] = 2*qx*qz - 2*qy*qw
    m[2][1] = 2*qy*qz + 2*qx*qw
    m[2][2] = 1 - 2*qx*qx - 2*qy*qy

    return m

def main():
    pygame.init()
    display = (800, 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL)

    gluPerspective(45, (display[0] / display[1]), 0.1, 50.0)
    glTranslatef(0.0, 0.0, -7)

    init_lighting()

    clock = pygame.time.Clock()
    current_q = [1, 0, 0, 0]

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False

        q = read_quaternion()
        if q:
            current_q = q

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glPushMatrix()

        # Load quaternion rotation
        rot_matrix = quaternion_to_matrix(current_q)
        glMultMatrixf(rot_matrix.T)  # transpose for OpenGL column-major

        draw_cube()
        glPopMatrix()

        pygame.display.flip()
        clock.tick(60)

    ser.close()
    pygame.quit()

if __name__ == "__main__":
    main()
